# Cluster Expansion for Ni-Al with ICET

This tutorial demonstrates building a cluster expansion for the Ni-Al binary system using ICET and VASP.

In [ ]:
import numpy as np
from ase import Atoms
from ase.build import bulk

# Check for ICET
try:
    from icet import ClusterSpace, StructureContainer, ClusterExpansion
    from icet.tools import enumerate_structures
    from icet.tools.structure_generation import generate_sqs
    HAS_ICET = True
except ImportError:
    HAS_ICET = False
    print("ICET not installed. Run: pip install icet")

from vasp import Vasp
from vasp.runners import MockRunner, MockResults

## Step 1: Define the Parent Lattice

The parent lattice is FCC with sites that can be occupied by Ni or Al.

In [ ]:
# FCC primitive cell as parent lattice
a = 3.5  # Approximate lattice constant
parent = bulk('Ni', 'fcc', a=a)

print(f"Parent lattice: {parent.get_chemical_formula()}")
print(f"Lattice constant: {a} Å")
print(f"Atoms in primitive cell: {len(parent)}")

## Step 2: Create Cluster Space

Define which clusters to include in the expansion.

In [ ]:
if HAS_ICET:
    # Define cluster space
    # cutoffs: [pair, triplet] in Angstrom
    cutoffs = [5.0, 4.0]  # Include pairs up to 5Å, triplets up to 4Å
    
    cs = ClusterSpace(
        structure=parent,
        cutoffs=cutoffs,
        chemical_symbols=['Ni', 'Al']
    )
    
    print(cs)
else:
    print("Skipping - ICET not installed")

## Step 3: Create Training Structures

We create a minimal training set with 5 structures.

In [ ]:
def create_training_structures():
    """Create minimal training set for Ni-Al."""
    structures = []
    
    # 1. Pure Ni (FCC)
    ni_pure = bulk('Ni', 'fcc', a=3.52, cubic=True)
    structures.append(('Ni', ni_pure, -5.50))  # Energy in eV/atom
    
    # 2. Pure Al (FCC)
    al_pure = bulk('Al', 'fcc', a=4.05, cubic=True)
    structures.append(('Al', al_pure, -3.74))  # Energy in eV/atom
    
    # 3. NiAl (B2 / CsCl structure)
    nial_b2 = Atoms(
        'NiAl',
        positions=[[0, 0, 0], [0.5*2.88, 0.5*2.88, 0.5*2.88]],
        cell=[2.88, 2.88, 2.88],
        pbc=True
    )
    structures.append(('NiAl_B2', nial_b2, -4.85))
    
    # 4. Ni3Al (L12 structure)
    ni3al = Atoms(
        'Ni3Al',
        positions=[
            [0, 0, 0],           # Al at corner
            [0.5*3.57, 0.5*3.57, 0],  # Ni at face centers
            [0.5*3.57, 0, 0.5*3.57],
            [0, 0.5*3.57, 0.5*3.57],
        ],
        symbols=['Al', 'Ni', 'Ni', 'Ni'],
        cell=[3.57, 3.57, 3.57],
        pbc=True
    )
    structures.append(('Ni3Al_L12', ni3al, -5.02))
    
    # 5. NiAl3 (L12 structure, Al-rich)
    nial3 = Atoms(
        'NiAl3',
        positions=[
            [0, 0, 0],           # Ni at corner
            [0.5*3.95, 0.5*3.95, 0],  # Al at face centers
            [0.5*3.95, 0, 0.5*3.95],
            [0, 0.5*3.95, 0.5*3.95],
        ],
        symbols=['Ni', 'Al', 'Al', 'Al'],
        cell=[3.95, 3.95, 3.95],
        pbc=True
    )
    structures.append(('NiAl3_L12', nial3, -3.95))
    
    return structures

training_set = create_training_structures()

print("Training structures:")
print("-" * 50)
for name, atoms, energy in training_set:
    n_ni = sum(1 for s in atoms.symbols if s == 'Ni')
    n_al = sum(1 for s in atoms.symbols if s == 'Al')
    x_al = n_al / len(atoms)
    print(f"{name:12s}  x_Al={x_al:.2f}  E={energy:.2f} eV/atom")

## Step 4: Calculate DFT Energies with VASP

In practice, you would run VASP for each structure. Here we use mock energies.

In [ ]:
def calculate_energy(atoms, label, mock_energy):
    """Calculate energy with VASP (using mock for demonstration)."""
    
    # Mock results (in real use, remove mock and run actual VASP)
    mock = MockResults(
        energy=mock_energy * len(atoms),
        forces=np.zeros((len(atoms), 3)),
    )
    
    calc = Vasp(
        label=f'ce_train/{label}',
        atoms=atoms,
        runner=MockRunner(results=mock),
        xc='PBE',
        encut=400,
        kpts=(8, 8, 8),
        ismear=1,
        sigma=0.1,
    )
    
    return calc.potential_energy / len(atoms)  # eV/atom

# Calculate energies
dft_energies = []
for name, atoms, mock_e in training_set:
    e = calculate_energy(atoms, name, mock_e)
    dft_energies.append((atoms, e))
    print(f"{name}: E = {e:.4f} eV/atom")

## Step 5: Fit the Cluster Expansion

In [ ]:
if HAS_ICET:
    from trainstation import CrossValidationEstimator
    
    # Build structure container
    sc = StructureContainer(cluster_space=cs)
    
    for (name, atoms, _), (_, energy) in zip(training_set, dft_energies):
        sc.add_structure(
            structure=atoms,
            user_tag=name,
            properties={'energy': energy}
        )
    
    print(f"Structure container: {len(sc)} structures")
    print(sc)
else:
    print("Skipping - ICET not installed")

In [ ]:
if HAS_ICET:
    # Fit with cross-validation
    opt = CrossValidationEstimator(
        fit_data=sc.get_fit_data(key='energy'),
        fit_method='lasso',
        validation_method='shuffle-split',
        number_of_splits=3
    )
    opt.validate()
    opt.train()
    
    print(f"RMSE: {1000 * opt.rmse_train:.1f} meV/atom")
    
    # Create cluster expansion
    ce = ClusterExpansion(
        cluster_space=cs,
        parameters=opt.parameters
    )
    
    print("\nEffective Cluster Interactions (ECIs):")
    print(ce)
else:
    print("Skipping - ICET not installed")

## Step 6: Generate SQS Structure

A Special Quasirandom Structure (SQS) mimics a random alloy in a small supercell.

In [ ]:
if HAS_ICET:
    from icet.tools.structure_generation import generate_sqs
    
    # Generate SQS for Ni0.5Al0.5 (50% each)
    target_concentrations = {'Ni': 0.5, 'Al': 0.5}
    
    # Use 2x2x2 supercell (32 atoms)
    supercell = parent.repeat((2, 2, 2))
    
    sqs = generate_sqs(
        cluster_space=cs,
        max_size=len(supercell),
        target_concentrations=target_concentrations,
        n_steps=5000,
    )
    
    print(f"SQS structure: {sqs.get_chemical_formula()}")
    print(f"Number of atoms: {len(sqs)}")
    print(f"Composition: Ni={sum(s=='Ni' for s in sqs.symbols)/len(sqs):.2f}")
    
    # Predict energy with cluster expansion
    e_sqs = ce.predict(sqs)
    print(f"Predicted energy: {e_sqs:.4f} eV/atom")
else:
    print("Skipping - ICET not installed")
    print("\nManual SQS example (without ICET):")
    
    # Create a simple disordered structure manually
    sqs_manual = bulk('Ni', 'fcc', a=3.5, cubic=True).repeat((2, 2, 2))
    # Replace half with Al randomly
    symbols = list(sqs_manual.symbols)
    n_replace = len(symbols) // 2
    indices = np.random.choice(len(symbols), n_replace, replace=False)
    for i in indices:
        symbols[i] = 'Al'
    sqs_manual.set_chemical_symbols(symbols)
    
    print(f"Manual SQS: {sqs_manual.get_chemical_formula()}")
    print(f"Atoms: {len(sqs_manual)}")

## Step 7: Calculate SQS Energy with VASP

Use the SQS structure for a VASP calculation to get the random alloy energy.

In [ ]:
# For demonstration, use the manual SQS
sqs_structure = sqs_manual if not HAS_ICET else sqs

# Mock energy for random alloy (interpolation + mixing energy)
# E_mix ≈ -0.1 eV/atom for Ni-Al at 50-50
e_ni, e_al = -5.50, -3.74
e_mix = 0.5 * e_ni + 0.5 * e_al - 0.10  # Simple estimate

mock_sqs = MockResults(
    energy=e_mix * len(sqs_structure),
    forces=np.zeros((len(sqs_structure), 3)),
)

calc_sqs = Vasp(
    label='ce_train/sqs_Ni50Al50',
    atoms=sqs_structure,
    runner=MockRunner(results=mock_sqs),
    xc='PBE',
    encut=400,
    kpts=(4, 4, 4),  # Fewer k-points for larger cell
    ismear=1,
    sigma=0.1,
)

e_sqs_dft = calc_sqs.potential_energy / len(sqs_structure)
print(f"SQS DFT energy: {e_sqs_dft:.4f} eV/atom")
print(f"Reference (linear interpolation): {0.5*e_ni + 0.5*e_al:.4f} eV/atom")
print(f"Mixing energy: {e_sqs_dft - 0.5*e_ni - 0.5*e_al:.4f} eV/atom")

## Summary

This example demonstrated:

1. **Cluster space setup** - Define parent lattice and cutoffs
2. **Training structures** - Pure elements + ordered compounds (5 structures)
3. **DFT calculations** - VASP energies for training
4. **CE fitting** - LASSO regression with cross-validation
5. **SQS generation** - Random alloy structure for 50-50 composition

### For Production Use

- Use more training structures (20-100)
- Include relaxed structures
- Validate CE predictions with DFT
- Use Monte Carlo for thermodynamics

In [ ]:
# Save SQS structure for VASP calculation
from ase.io import write

write('sqs_Ni50Al50.vasp', sqs_structure, format='vasp')
print("Saved: sqs_Ni50Al50.vasp")